# 1. The ```plc.Planet``` class

Since Pylightcurve 4.0.0 we can access many core calculations related to exoplanets through the ```plc.Planet``` object.

## 1.1 Automated definition throught the ExoClock database

We can quickly create a ```plc.Planet``` object based on catalague data as follows:

In [ ]:
%matplotlib notebook

import numpy as np
import matplotlib.pyplot as plt
import pylightcurve as plc

planet = plc.get_planet('hd209458b')


At the moment the data provided are base on the Exoplanet Characterisation Catalogue (ECC) developed as part of the [```ExoClock Project```](https://www.exoclock.space) ([Kokori et al. 2021](https://link.springer.com/article/10.1007/s10686-020-09696-3)). The catalogue contains about 750 objects and will gradually expand in future releases.

To retrieve a list of all the available planet names we can type:


In [ ]:
plc.get_all_planets()

## 1.2 Manual definition

Alternatively, we can define a ```plc.Planet``` object manually:


In [ ]:
planet = plc.Planet(
    name='HD209458b',              # str
    ra = 330.795,                  # float, in degrees
    dec = 18.884,                  # float, in degrees
    stellar_logg = 4.36,           # float, in log(cm/s^2)
    stellar_temperature = 6065.0,  # float, in Kelvin
    stellar_metallicity = 0.0,     # float, in dex(Fe/H) or dex(M/H)
    rp_over_rs = 0.12086,          # float, no units
    period = 3.5247486,            # float, in days
    sma_over_rs = 8.76,            # float, no units
    eccentricity = 0.0,            # float, no units
    inclination = 86.71,           # float, in degrees          
    periastron = 0.0,              # float, in degrees
    mid_time = 2452826.62928,      # float, in days (BJD-TDB)  
    ldc_method = 'claret',         # str, available methods: claret(default), power2, linear, quad
    max_sub_exp_time=10,           # float, default = 10 seconds
    precision=2,                   # int 0 to 10, default = 2
    asc_node=0,                    # float, default = 0.0 degrees
    albedo = 0.15,                 # float, default = 0.15, no units 
    emissivity = 1.0,              # float, default = 1.0, no units
)

**Note**: if we have the ```mid_time``` in a format different from BJD<sub>TDB</sub> (e.g. HJD<sub>UTC</sub>) we need to convert it  before passing it to the planet object:
``` python
mid_time = plc.convert_to_bjd_tdb(ra, dec, mid_time_in_hjd_utc, 'HJD_UTC')
```

## 1.3 Definition of parameters


| variable name            | type       | decription                                |unit                    |
|:-------------------------|:----------:|:-----------------------------------------:|:----------------------:|
| ``name``                 |``str``     |name of the planet                         |          --            |
| ``ra``                   |``float``   |RA of the host star                        |degrees (ICRS)          |
| ``dec``                  |``float``   |DEC of the host star                       |degrees (ICRS)          |
| ``stellar_logg``         |``float``   |log(g) of the host star                    |log(cm/s<sup>2)         |
| ``stellar_temperature``  |``float``   |effecive temperature of the host star      |Kelvin                  |
| ``stellar_metallicity``  |``float``   |metallicity of the host star               |dex(Fe/H) or dex(M/H)   |
| ``rp_over_rs``           |``float``   |plant-to-star radius ratio                 |          --            |
| ``period``               |``float``   |orbital period                             |days                    |
| ``sma_over_rs``          |``float``   |orbital semi-major axis relatively <br> to the stellar radius       |          --            |
| ``eccentricity``         |``float``   |orbital eccentricity                       |          --            |
| ``inclination``          |``float``   |orbital inclination                        |degrees                 |
| ``periastron``           |``float``   |orbital argument of periastron             |degrees                 |
| ``mid_time``             |``float``   |the time of conjunction with the planet in front of the star |days (BJD<sub>TDB</sub>)|



### ```ldc_method = 'claret'```

This parameter refers to the limb-darkening law that we want to use. The default value is ```claret```, and the other options are: ```power2```, ```linear``` and ```quad```.


### ```max_sub_exp_time = 10```
   
This parameter defines the time-resolution of the integrated transit models (see below for what the intgated model is) that will be calculated for this planet, in seconds. The default value is 10, meaning that the integrated models with exposure times below 10 seconds will be identical. 
    
    
### ```precision = 2```

This parameter is an option for the functions that calculate the relative flux during a transit or an eclipse. It refers to the precision of the numerical calculation of the Gauss–Kronrod quadrature, the default value is 2, and the other options are 3, 4, ... 10, meaning that 20, 30, 40, ... 100 quadrature points will be used.

We can check the balance between precision and speed (to adjust the precision level to our needs), as follows:  


In [ ]:
planet.performance_report(array_size=100)

### ```asc_node = 0```
    
This is the longitude of the scending node in degrees. This parameter doesn't affect the transit or eclipse depth, and by default is 0.

### ```albedo = 0.15```, ```emissivity = 1.0```
    
These two parameters have the above dafault values (no units) and they affect the theoretical calculation of the expected eclipse depth, as they are used to calculate the planetary temperature as folows:
 
```python
planet_temperature = stellar_temperature * np.sqrt(0.5 / sma_over_rs) * (((1 - albedo) / emissivity) ** 0.25)
```

## 1.4 PyLightcurve core calculations though a ```planet``` object

Through a ```plc.Planet``` object we can quickly access the core calculations of Pylightcurve 
These calulations are:

- **```planet.exotethys```**: calculates the limb-darkening coefficients
- **```planet.fp_over_fs```**: calculates the planet-to-star flux ratio (flux emmitted + reflected by the planet)
- **```planet.convert_to_bjd_tdb```**: converts a time array from different time formats to BJD<sub>TDB</sub>
- **```planet.convert_to_jd_utc```**: converts a time array from different time formats to JD<sub>UTC</sub>
- **```planet.convert_to_relflux```**: converts a flux array from magnitude to relative flux
- **```planet.planet_orbit```**: calculates the (x,y,z) position of the planet in a cartesian frame with the star at (0,0,0), the Earth at (+infinity,0,0) and the periastron located on the −y axis for $\omega$=0.
- **```planet.planet_star_projected_distance```**: calculates the distance between the planet and the star relatively to the stellar radius, as projected on the celestial sphere.
- **```planet.planet_phase```**: calculates the planet phase with respect to the transit mid-time (time of conjunction when the planet is behind the star, no units)
- **```planet.transit_depth```**: calculates the transit depth (in relative flux)
- **```planet.transit_duration```**: calculates the transit duration (in days)
- **```planet.transit_t12```**: calculates the time between transit contact points 1 and 2 (in days)
- **```planet.eclipse_mid_time```**: calculates the eclipse mid-time (time of conjunction when the planet is behind the star, in days BJD<sub>TDB</sub>)
- **```planet.eclipse_depth```**: calculates the eclipse depth (in relative flux)
- **```planet.eclipse_duration```**: calculates the eclipse duration (in days)
- **```planet.transit```**: calculates the transit model (in relative flux)
- **```planet.eclipse```**: calculates the eclipse model (in relative flux)
- **```planet.transit_integrated```**: calculates the exposure-integtared transit model (in relative flux)
- **```planet.eclipse_integrated```**: calculates the exposure-integtared eclipse model (in relative flux)



### 1.4.1 ```planet.exotethys``` 

*Calculates the limb-darkening coefficients.*

PyLightcurve includes a wrapper for the [```ExoTETHyS```](https://github.com/ucl-exoplanets/ExoTETHyS) package ([Morello et al. 2020](https://iopscience.iop.org/article/10.3847/1538-3881/ab63dc)), to allow for the easy calclation of the limb-darkening coefficients.

Returns a 4-element array. If the limb-darkening law used for the ```plc.Planet``` object is not ```claret```, the extra elements will be zero.

**Example 1**:

In [ ]:
planet.exotethys(filter_name = 'COUSINS_R',)

**Example 2**:

We may want to use different stellar models to calculate the limb-darkening coefficients
There is a range of stellar models availabe throught the [```ExoTETHyS```](https://github.com/ucl-exoplanets/ExoTETHyS) package pachage.
The default stellar model is the ```Phoenix_2018```. However some stellar parameters and some 
wavelengths, are not covered by all models. For example, ```Phoenix_2018``` are not availble 
for the ```irac4``` filter, or for very cool stars (e.g. 3000 K).
The other stellar models are: ```Atlas_2000```, ```Phoenix_2012_13```, ```Phoenix_drift_2012```, ```Stagger_2015```, ```Stagger_2018```.
Please consult the the [```ExoTETHyS```](https://github.com/ucl-exoplanets/ExoTETHyS) package page if an error occurs.
    

In [ ]:
planet.exotethys(filter_name = 'irac4', stellar_model='Atlas_2000')

**Example 3**:

For the cases where a specroscopic band-pass is needed, we can also specify a part of the filter's bandpass to be used, by defining the ```wlrange``` parameter (by default is ```None```). This is a list with two elements, indicating the limits (in Angstrom) within the filter bandpass that we want to use. The wavelength range needs to be within the limits of the filter bandpass, otherwise the calsulation will fail.


In [ ]:
planet.exotethys(filter_name = 'hst_wfc3_ir_g141', wlrange = [11000, 12000])

### 1.4.2 ```planet.fp_over_fs``` 

*Calculates the planet-to-star flux ratio (flux emmitted + reflected by the planet).*

Here, the ```filter_name``` and ```wlrange``` parameters can be used in the same way as in the ```planet.exotethys``` funcion.

In [ ]:
planet.fp_over_fs(filter_name='COUSINS_R')

In [ ]:
planet.fp_over_fs(filter_name = 'hst_wfc3_ir_g141', wlrange = [11000, 12000])

### 1.4.3 ```planet.convert_to_bjd_tdb```

*Converts a time array from different time formats to BJD<sub>TDB</sub>.*

```time_format``` defines the format of the input time-series, and it can be: 
- ```JD_UTC```
- ```MJD_UTC```
- ```BJD_UTC```
- ```BJD_TDB```
- ```BJD_TT```
- ```HJD_UTC```
- ```HJD_BJD```
- ```HJD_TT```

In [ ]:
time_array_in_jd_utc = np.arange(2460000.0, 2460010.0, 0.1)

time_array_in_bjd_tdb = planet.convert_to_bjd_tdb(time_array_in_jd_utc, time_format='JD_UTC')

plt.figure(figsize=(7, 5))
plt.plot(
    time_array_in_bjd_tdb, 
    (time_array_in_bjd_tdb - time_array_in_jd_utc) * (24 * 60), 'ko')
plt.ylabel(r'BJD$_\mathrm{TDB}$ - JD$_\mathrm{UTC}$ (min)')
plt.xlabel(r'Time (BJD$_\mathrm{TDB}$)')

### 1.4.4 ```planet.convert_to_jd_utc```

*Converts a time array from different time formats to JD<sub>UTC</sub>.*

```time_format``` defines the format of the input time-series, and it can be: 
- ```JD_UTC```
- ```MJD_UTC```
- ```BJD_UTC```
- ```BJD_TDB```
- ```BJD_TT```
- ```HJD_UTC```
- ```HJD_BJD```
- ```HJD_TT```

In [ ]:
time_array_in_bjd_tdb = np.arange(2460000.0, 2460010.0, 0.1)

time_array_in_jd_utc = planet.convert_to_jd_utc(time_array_in_bjd_tdb, time_format='BJD_TDB')

plt.figure(figsize=(7, 5))
plt.plot(
    time_array_in_bjd_tdb, 
    (time_array_in_bjd_tdb - time_array_in_jd_utc) * (24 * 60), 'ko')
plt.ylabel(r'BJD$_\mathrm{TDB}$ - JD$_\mathrm{UTC}$ (min)')
plt.xlabel(r'Time (BJD$_\mathrm{TDB}$)')

### 1.4.5 ```planet.convert_to_relflux```

*Converts a flux array from magnitude to relative flux.*

```flux_format``` defines the format of the input series, and it can be: 
- ```flux```
- ```mag```

In [ ]:
planet.convert_to_relflux(np.array([9, 9.1, 9.2]), np.array([0.01, 0.01, 0.01]), 'mag')

### 1.4.6 ```planet.planet_orbit```

*Calculates the (x,y,z) position of the planet in a cartesian frame with the star at (0,0,0), the Earth at (+infinity,0,0) and the periastron located on the −y axis for $\omega$=0.*

In [ ]:
time_array = np.arange(planet.mid_time - 1.5 * planet.period, planet.mid_time + 1.5 * planet.period, 0.01)

x, y, z = planet.planet_orbit(time_array)

plt.figure()
plt.plot(time_array, x, 'ko', ms=2, label='x')
plt.plot(time_array, y, 'ro', ms=2, label='y')
plt.plot(time_array, z, 'bo', ms=2, label='z')
plt.legend()
plt.ylabel('Planet-star distance / $R_*$')
plt.xlabel(r'Time (BJD$_\mathrm{TDB}$)')

### 1.4.7 ```planet.planet_star_projected_distance```

*Calculates the distance between the planet and the star relatively to the stellar radius, as projected on the celestial sphere.*

In [ ]:
time_array = np.arange(planet.mid_time - 1.5 * planet.period, planet.mid_time + 1.5 * planet.period, 0.01)

planet_star_projected_distance = planet.planet_star_projected_distance(time_array)


plt.figure()
plt.plot(time_array, planet_star_projected_distance, 'ko', ms=2)
plt.ylabel('Planet-star projected distance / $R_*$')
plt.xlabel(r'Time (BJD$_\mathrm{TDB}$)')

### 1.4.8 ```planet.planet_phase```

*Calculates the planet phase with respect to the transit mid-time (time of conjunction when the planet is in front of the star, no units).*

In [ ]:
time_array = np.arange(planet.mid_time - 1.5 * planet.period, planet.mid_time + 1.5 * planet.period, 0.01)

planet_phase = planet.planet_phase(time_array)


plt.figure()
plt.plot(time_array, planet_phase, 'ko', ms=2)
plt.ylabel('Planet phase')
plt.xlabel(r'Time (BJD_$\mathrm{TDB}$)')

### 1.4.9 ```planet.transit_depth```

*Calculates the transit depth (in relative flux).*

Here, the ```filter_name``` and ```wlrange``` parameters can be used in the same way as in the ```planet.exotethys``` funcion.

In [ ]:
planet.transit_depth(filter_name='COUSINS_R')

In [ ]:
planet.transit_depth(filter_name = 'hst_wfc3_ir_g141', wlrange = [11000, 12000])

### 1.4.10 ```planet.transit_duration```

*Calculates the transit duration (in days).*

In [ ]:
planet.transit_duration()

### 1.4.11 ```planet.transit_t12```

*Calculates the time between transit contact points 1 and 2 (in days).*

In [ ]:
planet.transit_t12()

### 1.4.12 ```planet.eclipse_mid_time```

*Calculates the eclipse mid-time (time of conjunction when the planet is behind the star, in days BJD<sub>TDB</sub>).*

In [ ]:
planet.eclipse_mid_time()

### 1.4.13 ```planet.eclipse_depth```

*Calculates the eclipse depth (in relative flux).*

Here, the ```filter_name``` and ```wlrange``` parameters can be used in the same way as in the ```planet.exotethys``` funcion.

In [ ]:
planet.eclipse_depth(filter_name='COUSINS_R')

In [ ]:
planet.eclipse_depth(filter_name = 'hst_wfc3_ir_g141', wlrange = [11000, 12000])

### 1.4.14 ```planet.eclipse_duration```

*Calculates the eclipse duration (in days).*

In [ ]:
planet.eclipse_duration()

### 1.4.15 ```planet.transit```

*Calculates the transit model (in relative flux).*

Here, the ```filter_name``` and ```wlrange``` parameters can be used in the same way as in the ```planet.exotethys``` funcion.

In [ ]:
time_array = np.arange(planet.mid_time - 1.1 * planet.transit_duration(), 
                       planet.mid_time + 1.1 * planet.transit_duration(), 0.001)

flux_jonhson_b = planet.transit(time_array, filter_name='JOHNSON_B')
flux_cousins_r = planet.transit(time_array, filter_name='COUSINS_R')

plt.figure()
plt.plot(time_array, flux_jonhson_b, 'o', ms=2, label='JOHNSON_B')
plt.plot(time_array, flux_cousins_r, 'ro', ms=2, label='COUSINS_R')
plt.ylabel('Flux (relative)')
plt.xlabel(r'Time (BJD$_\mathrm{TDB}$)')
plt.legend()


### 1.4.16 ```planet.eclipse```

*Calculates the eclipse model (in relative flux).*

Here, the ```filter_name``` and ```wlrange``` parameters can be used in the same way as in the ```planet.exotethys``` funcion.

In [ ]:
time_array = np.arange(planet.eclipse_mid_time() - 1.1 * planet.eclipse_duration(), 
                       planet.eclipse_mid_time() + 1.1 * planet.eclipse_duration(), 0.001)

flux_jonhson_b = planet.eclipse(time_array, filter_name='JOHNSON_B')
flux_cousins_r = planet.eclipse(time_array, filter_name='COUSINS_R')


plt.figure()
plt.plot(time_array, flux_jonhson_b, 'o', ms=2, label='JOHNSON_B')
plt.plot(time_array, flux_cousins_r, 'ro', ms=2, label='COUSINS_R')
plt.ylabel('Flux (relative)')
plt.xlabel(r'Time (BJD$_\mathrm{TDB}$)')
plt.legend()

### 1.4.17 ```planet.transit_integrated```

*Calculates the exposure-integtared transit model (in relative flux).*

```exp_time``` defines the exposure time of each dtaapoint in seconds. The time array mush indicate the mid-exposure time. 

Here, the ```filter_name``` and ```wlrange``` parameters can be used in the same way as in the ```planet.exotethys``` funcion.

In [ ]:
time_array = np.arange(planet.mid_time - 1.1 * planet.transit_duration(), 
                       planet.mid_time + 1.1 * planet.transit_duration(), 0.001)

flux_jonhson_b = planet.transit_integrated(time_array, exp_time=100, filter_name='JOHNSON_B')
flux_cousins_r = planet.transit_integrated(time_array, exp_time=100, filter_name='COUSINS_R')

plt.figure()
plt.plot(time_array, flux_jonhson_b, 'o', ms=2, label='JOHNSON_B')
plt.plot(time_array, flux_cousins_r, 'ro', ms=2, label='COUSINS_R')
plt.ylabel('Flux (relative)')
plt.xlabel(r'Time (BJD$_\mathrm{TDB}$)')
plt.legend()


In [ ]:
time_array = np.arange(planet.mid_time - 1.1 * planet.transit_duration(), 
                       planet.mid_time + 1.1 * planet.transit_duration(), 0.001)

flux_cousins_r_10s = planet.transit_integrated(time_array, exp_time=10, filter_name='COUSINS_R')
flux_cousins_r_100s = planet.transit_integrated(time_array, exp_time=100, filter_name='COUSINS_R')

plt.figure()
plt.plot(time_array, 1000000*(flux_cousins_r_10s - flux_cousins_r_100s), 'ko', ms=2)
plt.ylabel('Flux 10s exposure - 100s exposure (ppm)')
plt.xlabel(r'Time (BJD$_\mathrm{TDB}$)')
plt.legend()


### 1.4.18 ```planet.eclipse_integrated```

*Calculates the exposure-integtared eclipse model (in relative flux).*

```exp_time``` defines the exposure time of each dtaapoint in seconds. The time array mush indicate the mid-exposure time. 

Here, the ```filter_name``` and ```wlrange``` parameters can be used in the same way as in the ```planet.exotethys``` funcion.

In [ ]:
time_array = np.arange(planet.eclipse_mid_time() - 1.1 * planet.eclipse_duration(), 
                       planet.eclipse_mid_time() + 1.1 * planet.eclipse_duration(), 0.001)

flux_jonhson_b = planet.eclipse_integrated(time_array, exp_time=100, filter_name='JOHNSON_B')
flux_cousins_r = planet.eclipse_integrated(time_array, exp_time=100, filter_name='COUSINS_R')


plt.figure()
plt.plot(time_array, flux_jonhson_b, 'o', ms=2, label='JOHNSON_B')
plt.plot(time_array, flux_cousins_r, 'ro', ms=2, label='COUSINS_R')
plt.ylabel('Flux (relative)')
plt.xlabel(r'Time (BJD$_\mathrm{TDB}$)')
plt.legend()

## 1.5 Creating a simulated light-curve

In [ ]:
planet = plc.get_planet('WASP-12b')


In [ ]:
# define the filter
filter_name = 'COUSINS_R'

# define the out-of-transit observation time in hours
oot = 2

# define the exposure time in seconds
exp_time = 60

# define the noise (relatve flux)
noise = 0.0001



# calculate the transit duration 
transit_duration = planet.transit_duration()

# define the time array
time_array = np.arange(planet.mid_time - 0.5 * transit_duration - oot/24, 
                       planet.mid_time + 0.5 * transit_duration + oot/24, exp_time/60/60/24)

# calculate model
flux_array = planet.transit_integrated(time_array, exp_time=exp_time, filter_name=filter_name)

# add noise
flux_array += np.random.normal(0, noise, len(flux_array))

# calculate phase
phase_array = planet.planet_phase(time_array)



plt.figure()
plt.plot(phase_array, flux_array, 'ko', ms=2)
plt.ylabel('Flux (relative)')
plt.xlabel('Phase')
